In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET
import re
import xml.dom.minidom
import numpy as np
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET
import cv2
import glob

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
!wget https://gitlab.com/magistermilitum/ner_medieval_multilingual/-/raw/main/datasets/all_latin_texts_diplo.txt

In [ ]:
#orden para instalar las paginas xml en una carpeta pages
!wget https://gitlab.com/magistermilitum/e-ndp_htr/-/raw/main/page/all_manual_transcriptions.zip
zip_url="/content/all_manual_transcriptions.zip"

!mkdir all_manual_transcriptions
!unzip $zip_url -d /content/all_manual_transcriptions/

!wget https://gitlab.com/magistermilitum/e-ndp_htr/-/raw/main/page/endp_pages_all_V7.zip #14k PAGE-XML files
!mkdir all_transcriptions
zip_url="/content/endp_pages_all_V7.zip"
!unzip $zip_url -d /content/all_transcriptions/


In [ ]:
import json
import string
import unicodedata
import editdistance
import numpy as np


def ocr_metrics(predicts, ground_truth, norm_accentuation=False, norm_punctuation=False):
    """Calculate Character Error Rate (CER), Word Error Rate (WER) and Sequence Error Rate (SER)"""

    if len(predicts) == 0 or len(ground_truth) == 0:
        return (1, 1, 1)

    cer, wer, ser = [], [], []

    for (pd, gt) in zip(predicts, ground_truth):
        pd, gt = pd.lower(), gt.lower()

        if norm_accentuation:
            pd = unicodedata.normalize("NFKD", pd).encode("ASCII", "ignore").decode("ASCII")
            gt = unicodedata.normalize("NFKD", gt).encode("ASCII", "ignore").decode("ASCII")

        if norm_punctuation:
            pd = pd.translate(str.maketrans("", "", string.punctuation))
            gt = gt.translate(str.maketrans("", "", string.punctuation))

        pd_cer, gt_cer = list(pd), list(gt)
        #print(pd_cer)
        dist = editdistance.eval(pd_cer, gt_cer)
        cer.append(dist / (max(len(pd_cer), len(gt_cer))))

        pd_wer, gt_wer = pd.split(), gt.split()
        dist = editdistance.eval(pd_wer, gt_wer)
        wer.append(dist / (max(len(pd_wer), len(gt_wer))))

        pd_ser, gt_ser = [pd], [gt]
        dist = editdistance.eval(pd_ser, gt_ser)
        ser.append(dist / (max(len(pd_ser), len(gt_ser))))

    metrics = [cer, wer, ser]
    metrics = np.mean(metrics, axis=1)

    return round(list(metrics)[0], 4)

In [ ]:
#nube de puntos a coordenadas rectangulares
def square(h):
  h=list(map(int, h.replace(",", " ").split(" ")))
  h=list(zip(*[iter(h)]*2))
  h=np.array([h])
  h=cv2.boundingRect(h) #X coordinate, Y coordinate, width, height
  return h

In [ ]:
#guarda diccionario con id, puntos y contenido

nnn=[]
dict_unicodes={} #solo los textos
import xml.etree.ElementTree as ET

dict_registres={} #id, puntos y texto de cada linea
dict_regions={} #id, puntos y tipo de cada region

#for filename in glob.glob("/content/108_a/*.xml"):
#for filename in glob.glob("/content/all_transcriptions/raw_plus_manual/*.xml"):
for filename in glob.glob("/content/all_manual_transcriptions/*.xml"):
  nombre=filename.split("/")[-1].split(".")[0]
  tree=ET.parse(filename)
  root=tree.getroot()

  ids=[elem.attrib["id"] for elem in tree.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}TextLine')]
  points=[point.attrib["points"] for point in tree.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}Baseline')]
  nnn.append(points)

  unicodes=[uni.text for uni in tree.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}Unicode')]

  ids_regions=[elem.attrib["id"] for elem in tree.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}TextRegion')]
  #points_regions=[point for point in tree.findall(".//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}TextRegion")]

  points_regions=[]#coordenadas de cada region
  for point in tree.findall(".//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}TextRegion"):
    for y in point:
      try:
        points_regions.append(y.attrib["points"])
      except:
        continue


  #for address_ip in tree.find('sub2').find('source-IP').findall('address-IP'):


  type_regions=[]
  for elem  in tree.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}TextRegion'):
    try:
      type_regions.append(elem.attrib["custom"])
    except:
      type_regions.append("-")

  #dict_unicodes[nombre]=[x for x in unicodes if x!=None]
  xml_content=list(zip(ids, points, unicodes))#tupla: ids, baseline, text
  xml_regions=list(zip(ids_regions, points_regions, type_regions))

  if "5705" in filename:
    print(xml_content)

  dict_registres[nombre]=[[x[0], square(x[1]), x[2]] for x in xml_content if type(x[2])==str] #contiene: nombre de la linea, coordenadas square y texto de linea
  dict_regions[nombre]=[[x[0], square(x[1]), x[2]] for x in xml_regions if x[2]!="-" ]#contiene: nombre de la region, coordenadas square y typo de region

ordenada=sorted(dict_registres.keys())[35:]+sorted(dict_registres.keys())[:35]

dict_registres={k:dict_registres[k] for k in ordenada}

In [ ]:
# Function to transform XML extracted content to a single python dictionary
def transformation (filename):
  #print(filename)
  regions_list=["block", "Date", "liste", "entrée", "numérotation"] # the five trained regions

  dict_html={k:[] for k in regions_list} #dictionnary with regions as keys and list of text lines as values
  lista_html=[]

  for i, elem in enumerate(dict_registres[filename]): 
    # FOR THE LINES
    longueur=elem[1][0]+elem[1][2] #width starting point + size
    hauteur=elem[1][1]+elem[1][3] #height starting point + size
    central_point=(elem[1][0]+(elem[1][2]/2), elem[1][1]+round((elem[1][3]/3)), elem[1][1]+round((elem[1][3]/3))+round((elem[1][3]/3))) # squared line central point
   
    # FOR THE REGIONS
    region_sorted=sorted(dict_regions[filename], key=lambda l: l[1][2]-l[1][0])#some blocks are inside others
    for x in region_sorted:
      longueur_reg=x[1][0]+x[1][2] #width starting point + size
      hauteur_reg=x[1][1]+x[1][3] #height starting point + size
      central_point_reg=(x[1][0]+(x[1][2]/2), x[1][1]+(x[1][3]/2)) # squared region central point

     
      # Condition: We must find in wich region the central point of each line is localised
      if central_point[0]>x[1][0] and central_point[0]<x[1][0]+x[1][2] and central_point[1]>x[1][1] and central_point[1]<x[1][3]+x[1][1] and central_point[2]>x[1][1]:

      #if longueur<=longueur_reg and hauteur<=hauteur_reg:
        #print(x[2], "\n", elem, "\n\n" )
        dict_html[x[2].split("type:")[1].split(";")[0]].append(elem[2])
        lista_html.append([i, x[2].split("type:")[1].split(";")[0], elem[2]])
        break

  return lista_html, dict_html

In [ ]:
#Testing
#Warning: its mandatory use a deepcopy element in order to avoid loop modifications over iterations
import copy
a,b=transformation(list(dict_registres.keys())[10000])
c=copy.deepcopy(b)

In [ ]:
dict_registres["FRAN_0393_01725_L"]

In [ ]:
a,b=transformation(list(dict_registres.keys())[1])
d=copy.deepcopy(b)

In [ ]:
" ".join(c["block"])

In [ ]:
d["block"][3:12]

In [ ]:
1-ocr_metrics(c["block"][3:12], d["block"][3:12])

0.9829

In [ ]:
from collections import Counter
import nltk
nltk.download("punkt")
test=[]
diccionario=[]

from nltk.tokenize import word_tokenize


#with open("pretrain_src1_all_small.txt") as f:
with open("/content/all_latin_texts_diplo.txt") as f:
  f=f.readlines()
  f=[word_tokenize(x[:-1]) for x in f ]
  for x in f: diccionario.extend(x)
 

diccionario=Counter(diccionario)
diccionario_raw=[k for k,v in diccionario.items() if v>2]


In [ ]:
model.wv.most_similar("loquentur")

In [ ]:
for l in c["block"][3:12]:
  for w in word_tokenize(l):
    if w not in diccionario_raw and not w[0].isupper():
    
      liste=[x for x in model.wv.most_similar(w) if (x[0][0]==w[0] and x[1]>0.85 )]
      print(w, liste)

In [ ]:
gram=3
all_texts=[]
for k,v in dict_registres.items():
  text=[x[-1] for x in v if len(x[-1])>0]
  text=" ".join(text).split()
  text=[" ".join(text[i:i+gram]) for i in range(len(text)-gram+1)]
  all_texts.extend(text)

all_texts_txt="\n".join(all_texts)

with open("/content/all_transcribed.txt", 'w', encoding="utf-8") as f:
  f.write(all_texts_txt)

In [ ]:
len(all_texts)

In [ ]:
all_texts[:10]

In [ ]:
candidates

In [ ]:
from collections import Counter
#error="decembris suit festum"
error="festum betorum Petri"
candidates=[]

for s in all_texts:
  s=s.split()
  if fuzz.ratio(error.split()[0], s[0])>90 and fuzz.ratio(error.split()[2], s[2])>90 and fuzz.ratio(error.split()[1], s[1])>80:
    candidates.append(" ".join(s))

candidates=[(k,v) for k,v in Counter(candidates).items()]
candidates.sort(key=lambda x: x[1], reverse=True)
print(*candidates, sep="\n")

In [ ]:
from google.colab import drive

drive.mount("./gdrive", force_remount=True)

In [ ]:
from gensim.models import FastText
model = FastText.load_fasttext_format('/content/gdrive/MyDrive/Colab Notebooks/NER_multi/fasttext_multi/all_multi_fasttext_150dim_18_03_2022.bin')

In [ ]:
searched_word='alliquando'

liste=[x for x in model.wv.most_similar(searched_word) if x[0][0]==searched_word[0]]
liste

In [ ]:
import copy
import nltk
from collections import Counter
diccionario_trans=[]
for k,v in dict_registres.items():
  a,b=transformation(k)
  c=copy.deepcopy(b)
  c=" ".join(c["block"])
  c=word_tokenize(c)
  diccionario_trans.extend(c)
diccionario_trans=Counter(diccionario_trans)

In [ ]:
inter=(diccionario_raw^diccionario_trans.keys())&diccionario_trans.keys()
# operador ^ elimina los elementos comunes en ambos, es decir, existentes en el diccionario
#luego & busca la interseccion, es decir retiene solo los elementos de dict trans no existentes en dict raw.
inter={k:diccionario_trans[k] for k in list(inter)}
inter={k: v for k, v in sorted(inter.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
diccionario_trans_b=[]
for k,v in dict_registres.items():
  item=[]
  for line in v:
    if len(v)>2:
      item.append(line[-1])
  item=word_tokenize(" ".join(item))
  diccionario_trans_b.extend(item)

diccionario_trans_b=Counter(diccionario_trans_b)

In [ ]:
inter_b=(diccionario_trans_b.keys()^diccionario_trans.keys())&diccionario_trans.keys()
# operador ^ elimina los elementos comunes en ambos, es decir, existentes en el diccionario
#luego & busca la interseccion, es decir retiene solo los elementos de dict trans no existentes en dict raw.
inter_b={k:diccionario_trans[k] for k in list(inter_b)}
inter_b={k: v for k, v in sorted(inter_b.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
inter_b

In [ ]:
counter=0
for i,k in enumerate(inter_b.keys()):
  if i<2000:
    print(i, k, inter_b[k])
    counter+=inter_b[k]
print(counter/5)

In [ ]:
inter

In [ ]:
def count_intersections(lst1, lst2):
    c1 = Counter(lst1)
    c2 = Counter(lst2)
    return { k: min(c1[k], c2[k]) for k in c1.keys() & c2.keys() }

inter=count_intersections(diccionario_raw, diccionario_trans)

In [ ]:
diccionario_trans_b.keys()

dict_keys([])

In [ ]:
!pip install fasttext
import fasttext
#model = fasttext.train_unsupervised('/content/pretrain_src1_all_small.txt', minn=2, maxn=5, dim=100, epoch=10, verbose=1)
model = fasttext.train_unsupervised('/content/all_transcribed.txt', minn=2, maxn=5, dim=100, epoch=10, verbose=1)

In [ ]:
model.get_nearest_neighbors("decembris fuit festum")

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

tagged_data = [TaggedDocument(words=[x for x in word_tokenize(y)], tags=[str(i)]) for i,y in enumerate(all_texts)]


In [ ]:
#modeling vectors, this can take a while
max_epochs = 20
vec_size = 30
alpha = 0.025

model_d2v = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1, window=10, workers=4)
  
model_d2v.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.iter)
    # decrease the learning rate
    model_d2v.alpha -= 0.0002
    # fix the learning rate, no decay
    model_d2v.min_alpha = model_d2v.alpha

print("end modelization")

In [ ]:
model_d2v.save("encpos_doc2vec.model")

In [ ]:
model_d2v.most_similar(["decembris"])

In [ ]:
doc_embedding = model.encode([bloc])

In [ ]:
from collections import Counter
test=[]
diccionario=[]
#with open("pretrain_src1_all_small.txt") as f:
with open("training_txt_greek.txt") as f:
  f=f.readlines()
  f=[x[:-1] for x in f ]
  for x in f:
    if x.startswith("TE_L") or x.startswith("TE_P"):
      test.append(x.split(" ", 1)[1:])
    else:
      if x.startswith("VA_T"):
        continue
      else:
        diccionario.append(x.split(" ", 1)[1:])

test=[test[_:_+2] for _ in range(0, len(test), 2)]
diccionario=Counter([y for x in [x[0].split() for x in diccionario] for y in x])
diccionario_raw=[k for k,v in diccionario.items() if v>2]

In [ ]:
!pip install pie-extended

In [ ]:
!pie-extended download lasla
!pie-extended install-addons lasla

In [ ]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)
tagged_sentences=[]

sentences: List[str] = ["Lorem ipsum dolor sit amet, consectetur adipiscing elit. "]
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor
for sentence_group in sentences:
    iterator, processor = get_iterator_and_processor()
    tagged_sentences.extend([list(x.values())[:3] for x in tagger.tag_str(sentence_group, iterator=iterator, processor=processor)])
    #print(tagger.tag_str(sentence_group, iterator=iterator, processor=processor) )

In [ ]:
list(tagged_sentences[0].values())

In [ ]:
tagged_sentences

In [ ]:
import copy
import random
import json
import time
start_time = time.time()

tagged_sentences=[]

pages_list=list(dict_registres.keys())
dict_omnia_endp={}
nosketch_vertical="" # the vertical file is a classical column txt/not formatted file but supporting html tags

#with open('dict_omnia_endp_keywords.json', 'r') as json_file: dict_omnia_endp_keywords = json.load(json_file)

random.shuffle(pages_list) #optional

for i, k in enumerate(pages_list):
  if "FRAN_0393" in k:
    keywords_c="Unclassed"
    a,b=transformation(k)
    c=copy.deepcopy(b)
    #texto=" ".join(c["block"])
    
    page_id=int(k.split("_")[2].lstrip("0"))
    #print(numero, k)

    iterator, processor = get_iterator_and_processor()
    # The Omnia treetagger tagging
    for kk,vv in c.items(): #iteration occurs inside the content of each detected page region
      vv=" ".join(vv)
      if len(vv)>1:
        tagged_sentences.extend([list(x.values())[:3] for x in tagger.tag_str(vv, iterator=iterator, processor=processor)])
  

  if i%10==0:
    print(i)
    print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
tagged_sentences[:100]